In [57]:
import os
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import torch.nn as nn
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [58]:
data = pd.read_csv("..\\Data\\cle\\Rand_train.csv", sep=',')
data_val = pd.read_csv("..\\Data\\cle\\Rand_valid.csv", sep=',')
data_test = pd.read_csv("..\\Data\\cle\\Rand_test.csv", sep=',')

In [59]:
X_test = data_test.iloc[:,1:]
Y_test = data_test.iloc[:,0]

X2_test = X_test
Y2_test = Y_test

In [60]:
cut = 0

y1_test = (Y_test <= cut) * 1
y2_test = (Y_test > cut) * (-1) * (-Y_test)
Y2_test = np.stack((y1_test, y2_test), axis=-1)

In [61]:
cut0 = 0
cut1 = 1146
y10 = (Y_test == cut0) * 1
y11 = ((Y_test > cut0) & (Y_test <= cut1)) * 1
y12 = (Y_test > cut1) * 1
y21 = y11 * Y_test
y22 = y12 * Y_test
Y2_test = np.stack((y10, y11, y12, y21, y22), axis=-1)

In [62]:
class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        
        self.fc111 = nn.Linear(16, 24, bias = True)
        self.fc112 = nn.Linear(24, 1, bias = True)
        
        self.fc121 = nn.Linear(16, 24, bias = True)
        self.fc122 = nn.Linear(24, 1, bias = True)
        
        self.fc131 = nn.Linear(16, 24, bias = True)
        self.fc132 = nn.Linear(24, 1, bias = True)
       
        
        self.fc211 = nn.Linear(16, 24, bias=True)
        self.fc212 = nn.Linear(24, 5, bias=True)
        self.fc213 = nn.Linear(5, 1, bias=True)
        
        self.fc221 = nn.Linear(16, 24, bias=True)
        self.fc222 = nn.Linear(24, 5, bias=True)
        self.fc223 = nn.Linear(5, 1, bias=True)
        
        self.fc31 = nn.Linear(5,5, bias=False)       
        
               
    def softmax(self,x):
        expou = torch.exp(x)
        sm = expou/torch.sum(expou, dim=1).view(-1,1)
        return sm
        
    def disc_activation(self, h):
        v1 = self.softmax(h[:,:3])
        v2 = torch.exp(h[:,3:5])    
        return torch.cat([v1, v2], dim=1)
        
                
    def forward(self, x):
        x11 = torch.tanh(self.fc111(x))
        x11 = self.fc112(x11)
        
        x12 = torch.tanh(self.fc121(x))
        x12 = self.fc122(x12)
        
        x13 = torch.tanh(self.fc131(x))
        x13 = self.fc132(x13)                 
        
        #x21 = torch.tanh(self.fc211(x))
        x21 = self.fc211(x)
        x21 = F.relu(self.fc212(x21))
        x21 = self.fc213(x21)
        
        #x22 = torch.tanh(self.fc221(x))
        x22 = self.fc221(x)
        x22 = F.relu(self.fc222(x22))
        x22 = self.fc223(x22)
        
        x3 = torch.cat((x11, x12, x13, x21, x22), 1)        
        x3 = self.fc31(x3)
        x3 = self.disc_activation(x3)        
       
        return x3

In [63]:
PATH = '..\\Data\\checkpoints\\mytraining10.pt'
device = torch.device('cpu')
net = Net1()
net.load_state_dict(torch.load(PATH, map_location = device))

In [64]:
with torch.no_grad():
    net.eval()  
    
    test_inputs, test_labels = torch.FloatTensor(X2_test.values), torch.FloatTensor(Y2_test)
    test_inputs, test_labels = test_inputs.to(device), test_labels.to(device)      
    
    test_outputs = net.forward(test_inputs)   
    
    test_outputs2, test_labels2 = test_outputs.numpy(), test_labels.numpy()

In [65]:
y = test_labels2[:,1]
p = test_outputs2[:,0]
f2 = test_outputs2[:,1]
yhat = (1-p)*f2

In [66]:
y = test_labels2[:,2+1] + test_labels2[:,2+2]  
    
p1 = test_outputs2[:,1]
p2 = test_outputs2[:,2]
e1 = test_outputs2[:,2+1]
e2 = test_outputs2[:,2+2]    
yhat = p1 * e1 + p2 * e2

In [67]:
test_outputs2

array([[6.4894384e-01, 2.8141177e-01, 6.9644414e-02, 1.5368467e-02,
        1.0606888e+03],
       [7.1120042e-01, 2.4249856e-01, 4.6301045e-02, 1.0806758e-02,
        1.3455363e+03],
       [5.6609601e-01, 3.1945029e-01, 1.1445369e-01, 2.2067677e-02,
        7.5472845e+02],
       ...,
       [4.8175931e-01, 4.1142312e-01, 1.0681765e-01, 9.6489210e-03,
        1.4030319e+03],
       [5.1989760e-02, 4.1211876e-01, 5.3589141e-01, 1.3696923e-02,
        1.9840491e+03],
       [8.0987960e-01, 1.3201360e-01, 5.8106840e-02, 1.3120686e-01,
        1.5107277e+01]], dtype=float32)

In [68]:
print(np.mean(np.abs(yhat - y)))

511.9423


In [69]:
np.min(yhat)

0.33956465

In [70]:
np.max(yhat)

2514.3376

In [71]:
np.mean(yhat)

516.3637

In [72]:
cut0 = 0
cut1 = 1146

y1 = y[y==cut0]
yh1 = yhat[y==cut0]
y2 = y[(y > cut0) & (y <= cut1)]
yh2 = yhat[(y > cut0) & (y <= cut1)]
y3 = y[y>cut1]
yh3 = yhat[y>cut1]

In [73]:
print(np.mean(np.abs(yh1 - y1)))
print(np.mean(np.abs(yh2 - y2)))
print(np.mean(np.abs(yh3 - y3)))

234.59705
450.50894
2131.7993


In [56]:
10000000/2

5000000.0